In [1]:
import requests
import json
import pandas as pd
import datetime

from utils import formatURL
from config import getConfig

In [2]:
def getData(symbol, fromDt, toDt):
    url = baseURL + '&symbols=' + symbol + '&date_from='+ fromDt + '&date_to='+ toDt
    return requests.get(url).json()

In [3]:
def createDF(resp):
    # Input is the "get" response for a Symbol
    # Return a formatted df with the dates and prices
    dtList = []
    valList = []
    divList = []
    for row in resp['data']:
        dtList.append(row['date'][:10])
        valList.append(row['close'])
        divList.append(row['dividend'])
    d = {}
    d['date'] = dtList
    d['value'] = valList
    d['dividend'] = divList
    return pd.DataFrame(d).sort_values('date')

In [4]:
'''def getStartDt(maxDates, symbol):
    startDt = maxDates.get(symbol, None)
    if startDt:
        startDt =  startDt + datetime.timedelta(days=1)
    else:
        startDt = pd.to_datetime('2012-01-01').date()
    endDt = startDt + datetime.timedelta(days=365)
    return startDt, endDt'''

"def getStartDt(maxDates, symbol):\n    startDt = maxDates.get(symbol, None)\n    if startDt:\n        startDt =  startDt + datetime.timedelta(days=1)\n    else:\n        startDt = pd.to_datetime('2012-01-01').date()\n    endDt = startDt + datetime.timedelta(days=365)\n    return startDt, endDt"

In [4]:
cfg = getConfig()
baseURL = formatURL('eod')

In [6]:
# The dictionary will hold the most recent date where we have data for each fund
'''currentSymbols = set(df.fund)
maxDates = {}
for s in currentSymbols:
    maxDates[s] = df[df['fund']==s]['date'].max()'''

"currentSymbols = set(df.fund)\nmaxDates = {}\nfor s in currentSymbols:\n    maxDates[s] = df[df['fund']==s]['date'].max()"

In [5]:
symbol = 'GSPC.INDX'    # S&P 500
symbol = 'URTH'         # MSCI World

today = str(datetime.datetime.today().date())
apiResp = getData(symbol, '2020-01-01', '2024-01-01', )
df = createDF(apiResp)

In [6]:
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x).date())
df['year'] = df['date'].apply(lambda x: x.year)
df['month'] = df['date'].apply(lambda x: x.month)

In [7]:
# Data is daily: keep only the last day of the month
grp = df.groupby(['year', 'month'])['date'].max().values
df = df[df['date'].isin(grp)]
df = df[['date', 'value', 'dividend']]

In [8]:
#fileName = cfg['SPX']['dataLoc']+ cfg['SPX']['fileName']
fileName = cfg['MSCI']['dataLoc']+ cfg['MSCI']['fileName']
df.to_csv(fileName, index=False)

In [62]:
msciB = tmp

In [60]:
msciA['date'].min(), msciA['date'].max()

('2020-01-02', '2021-12-31')

In [63]:
msciB['date'].min(), msciB['date'].max()

('2022-01-03', '2023-12-29')

In [64]:
df = pd.concat([msciA, msciB])

In [70]:
fromDt, toDt = getStartDt(maxDates, symbol)
apiResp = getData(symbol, fromDt, toDt)

tmp = createDF(apiResp)
tmp['fund'] = symbol

dfList = []
dfList.append(tmp)
while toDt < datetime.date.today():
    fromDt = toDt + datetime.timedelta(days=1)
    toDt = toDt + datetime.timedelta(days=365)
    apiResp = getData(symbol, fromDt, toDt)
    tmp = createDF(apiResp)
    tmp['fund'] = symbol
    dfList.append(tmp)

upd = pd.concat(dfList).drop_duplicates().dropna()
upd = upd.sort_values(['fund', 'date']).reset_index(drop=True)
upd['date'] = upd['date'].apply(lambda x: pd.to_datetime(x).date())

df = pd.concat([df, upd])

In [71]:
# Check that all funds were updated
df.groupby('fund')['date'].max()

fund
990100.INDX       2022-07-15
ARKK              2022-08-11
CCJ               2022-08-11
COMB              2022-08-11
MSCIALL.INDX      2022-07-15
MSCIEAFE.INDX     2022-07-15
MSCIWORLD.INDX    2022-07-15
QYLD              2022-08-11
RYLD              2022-08-11
SP500TR.INDX      2022-08-12
Name: date, dtype: object

In [58]:
# Just in case I ran the same symbol twice
df = df.drop_duplicates()
# Fill NaN dividend with 0
df.fillna(0, inplace=True)

In [59]:
df.to_csv(cfg['dataLoc']+'fundData.csv', index=False)